# Equity Examples
(c) Marek Ozana

Example usage of polars-bloomberg in Equity space

In [12]:
# imports
import altair as alt
import polars as pl

from polars_bloomberg import BQuery


## S&P Annual Price Returns

In [67]:
# get S&P annual price returns
query = """
let(#px = px_last(dates=range(-100Y, 0D), fill=prev, per=Y);)
get(#px)
for('SPX Index')
"""
with BQuery() as bq:
    df = bq.bql(query).combine()

# Prepare data
bins: list = [
    "< -50",
    "-50 to -40",
    "-40 to -30",
    "-30 to -20",
    "-20 to -10",
    "-10 to 0",
    "0 to 10",
    "10 to 20",
    "20 to 30",
    "30 to 40",
    "40 to 50",
    "> 50",
]
g_data: pl.DataFrame = df.with_columns(
    pl.col("DATE").dt.year().alias("year"),
    pl.col("#px").pct_change().alias("rets"),
    pl.col("#px")
    .pct_change()
    .mul(100)
    .cut(
        range(-50, 60, 10),
        labels=bins,
    )
    .alias("bins"),
).drop_nulls()

# Create Chart
base = alt.Chart(g_data, title="S&P 500 Annual Price Returns").encode(
    x=alt.X("bins:O", title="Annual Returns (%)").sort(bins),
    y=alt.Y("count()", title="Count").stack("zero"),
)
bars = base.mark_bar().encode(
    color=alt.Color("year:O").scale(scheme="viridis").legend(None)
)
txt = base.mark_text(dy=9).encode(
    text=alt.Text("year:O"),
    color=alt.condition("datum.year < 1960", alt.value("white"), alt.value("black")),
)

(bars + txt).properties(width=600, height=400).configure_axisX(labelAngle=0)


alt.LayerChart(...)